In [1]:
# ============================================================
# STEP 1: Verify workspace exists
# ============================================================

import os
import shutil

COMPONENTS_DIR = './components/'
WORKSPACE_IMG  = './annotation_workspace/images/train'
WORKSPACE_LBL  = './annotation_workspace/labels/train'

os.makedirs(WORKSPACE_IMG, exist_ok=True)
os.makedirs(WORKSPACE_LBL, exist_ok=True)

# Copy images if workspace is empty
if len(os.listdir(WORKSPACE_IMG)) == 0:
    print("Copying images to workspace...")
    for f in os.listdir(COMPONENTS_DIR):
        if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            shutil.copy(os.path.join(COMPONENTS_DIR, f), WORKSPACE_IMG)

img_count = len([f for f in os.listdir(WORKSPACE_IMG) if f.lower().endswith(('.png','.jpg','.jpeg','.bmp'))])
lbl_count = len([f for f in os.listdir(WORKSPACE_LBL) if f.endswith('.txt')])

print(f"✓ Images:      {img_count}")
print(f"✓ Annotations: {lbl_count}")
print(f"✓ Remaining:   {img_count - lbl_count}")

✓ Images:      2556
✓ Annotations: 2197
✓ Remaining:   359


In [ ]:
# ============================================================
# STEP 2: Standalone Tkinter Annotation Tool - FIXED
# ============================================================

import tkinter as tk
from PIL import Image, ImageTk
import os

class AnnotationApp:
    def __init__(self, image_dir, label_dir):
        self.image_dir = image_dir
        self.label_dir = label_dir
        os.makedirs(label_dir, exist_ok=True)
        
        self.image_files = sorted([
            f for f in os.listdir(image_dir)
            if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))
        ])
        
        # Auto-resume
        self.current_idx = len(self.image_files)
        for i, f in enumerate(self.image_files):
            label_file = os.path.splitext(f)[0] + '.txt'
            if not os.path.exists(os.path.join(label_dir, label_file)):
                self.current_idx = i
                break
        
        self.box_start = None
        self.box_end = None
        self.dragging = False
        
        self.current_img = None
        self.display_img = None
        self.img_offset_x = 0
        self.img_offset_y = 0
        self.scale = 1.0
        
        self._build_ui()
        
        # FIXED: Wait for window to actually render, THEN load first image
        # root.after() schedules _load_image to run after the event loop starts
        self.root.after(100, self._load_image)
        
        self.root.mainloop()
    
    def _count_annotated(self):
        return sum(1 for f in self.image_files
                   if os.path.exists(os.path.join(self.label_dir,
                      os.path.splitext(f)[0] + '.txt')))
    
    def _build_ui(self):
        self.root = tk.Tk()
        self.root.title('Logo Annotation Tool')
        self.root.configure(bg='#1e1e1e')
        
        screen_w = self.root.winfo_screenwidth()
        screen_h = self.root.winfo_screenheight()
        win_w = min(900, screen_w - 100)
        win_h = min(950, screen_h - 50)
        self.root.geometry(f'{win_w}x{win_h}')
        self.root.resizable(True, True)
        
        # --- Progress label ---
        self.progress_var = tk.StringVar(value='Loading...')
        tk.Label(
            self.root, textvariable=self.progress_var,
            bg='#1e1e1e', fg='white', font=('Consolas', 11, 'bold')
        ).pack(side='top', pady=(8, 2))
        
        # --- Canvas ---
        canvas_frame = tk.Frame(self.root, bg='#1e1e1e')
        canvas_frame.pack(side='top', fill='both', expand=True, padx=10)
        
        self.canvas = tk.Canvas(canvas_frame, bg='#111111', highlightthickness=0)
        self.canvas.pack(fill='both', expand=True)
        
        self.canvas.bind('<ButtonPress-1>',   self._on_mouse_down)
        self.canvas.bind('<B1-Motion>',       self._on_mouse_drag)
        self.canvas.bind('<ButtonRelease-1>', self._on_mouse_up)
        
        # Keyboard
        self.root.bind('<s>', lambda e: self._save_and_next())
        self.root.bind('<S>', lambda e: self._save_and_next())
        self.root.bind('<n>', lambda e: self._skip())
        self.root.bind('<N>', lambda e: self._skip())
        self.root.bind('<p>', lambda e: self._previous())
        self.root.bind('<P>', lambda e: self._previous())
        
        # --- Buttons ---
        btn_frame = tk.Frame(self.root, bg='#1e1e1e')
        btn_frame.pack(side='bottom', pady=10)
        
        btn_kw = {'font': ('Consolas', 12, 'bold'), 'width': 16,
                  'relief': 'flat', 'cursor': 'hand2', 'bd': 0}
        
        tk.Button(btn_frame, text='✓  Save & Next  [S]',
                  bg='#2ecc71', fg='white', activebackground='#27ae60', activeforeground='white',
                  command=self._save_and_next, **btn_kw
        ).pack(side='left', padx=8)
        
        tk.Button(btn_frame, text='⏭  Skip  [N]',
                  bg='#e67e22', fg='white', activebackground='#d35400', activeforeground='white',
                  command=self._skip, **btn_kw
        ).pack(side='left', padx=8)
        
        tk.Button(btn_frame, text='◀  Previous  [P]',
                  bg='#3498db', fg='white', activebackground='#2980b9', activeforeground='white',
                  command=self._previous, **btn_kw
        ).pack(side='left', padx=8)
        
        # --- Status bar ---
        self.status_var = tk.StringVar(value='Loading...')
        tk.Label(
            self.root, textvariable=self.status_var,
            bg='#2c2c2c', fg='#aaaaaa', font=('Consolas', 9)
        ).pack(side='bottom', fill='x')
    
    # ----------------------------------------------------------
    # Mouse handlers
    # ----------------------------------------------------------
    
    def _on_mouse_down(self, event):
        self.box_start = (event.x, event.y)
        self.box_end = None
        self.dragging = True
        self._redraw_canvas()
    
    def _on_mouse_drag(self, event):
        if self.dragging:
            self.box_end = (event.x, event.y)
            self._redraw_canvas()
    
    def _on_mouse_up(self, event):
        self.dragging = False
        if self.box_start and self.box_end:
            if abs(self.box_end[0] - self.box_start[0]) < 5 or \
               abs(self.box_end[1] - self.box_start[1]) < 5:
                self.box_start = None
                self.box_end = None
                self._redraw_canvas()
                return
            self.status_var.set('Box drawn!  Press S or click "Save & Next"')
        self._redraw_canvas()
    
    # ----------------------------------------------------------
    # Image loading
    # ----------------------------------------------------------
    
    def _load_image(self):
        if self.current_idx >= len(self.image_files):
            self.progress_var.set('🎉  ALL DONE!')
            self.status_var.set(f'All {len(self.image_files)} images annotated.')
            self.canvas.delete('all')
            return
        
        img_file = self.image_files[self.current_idx]
        self.current_img = Image.open(os.path.join(self.image_dir, img_file)).convert('RGB')
        
        # Progress text
        done = self._count_annotated()
        total = len(self.image_files)
        self.progress_var.set(
            f'[{self.current_idx+1}/{total}]  {img_file}  |  {done}/{total} done ({done/total*100:.0f}%)'
        )
        
        # Reset box
        self.box_start = None
        self.box_end = None
        
        # Get actual canvas size (guaranteed valid here because of after())
        self.root.update_idletasks()
        canvas_w = self.canvas.winfo_width()
        canvas_h = self.canvas.winfo_height()
        
        # Safety check - fallback if still 0 for some reason
        if canvas_w <= 1 or canvas_h <= 1:
            canvas_w = 800
            canvas_h = 800
        
        # Fit image into canvas keeping aspect ratio
        img_w, img_h = self.current_img.size
        self.scale = min(canvas_w / img_w, canvas_h / img_h)
        
        new_w = max(1, int(img_w * self.scale))
        new_h = max(1, int(img_h * self.scale))
        
        self.img_offset_x = (canvas_w - new_w) // 2
        self.img_offset_y = (canvas_h - new_h) // 2
        
        self.display_img = self.current_img.resize((new_w, new_h), Image.LANCZOS)
        
        # Load existing annotation if present
        label_file = os.path.splitext(img_file)[0] + '.txt'
        label_path = os.path.join(self.label_dir, label_file)
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                parts = f.readline().strip().split()
                if len(parts) == 5:
                    _, xc, yc, w, h = map(float, parts)
                    x1 = int((xc - w/2) * img_w * self.scale + self.img_offset_x)
                    y1 = int((yc - h/2) * img_h * self.scale + self.img_offset_y)
                    x2 = int((xc + w/2) * img_w * self.scale + self.img_offset_x)
                    y2 = int((yc + h/2) * img_h * self.scale + self.img_offset_y)
                    self.box_start = (x1, y1)
                    self.box_end   = (x2, y2)
            self.status_var.set('Already annotated. Draw new box to overwrite, or press N to skip.')
        else:
            self.status_var.set('Click and drag on the image to draw a box around the logo')
        
        self._redraw_canvas()
    
    # ----------------------------------------------------------
    # Canvas drawing
    # ----------------------------------------------------------
    
    def _redraw_canvas(self):
        self.canvas.delete('all')
        
        if self.display_img is None:
            return
        
        # Base image
        self.tk_img = ImageTk.PhotoImage(self.display_img)
        self.canvas.create_image(self.img_offset_x, self.img_offset_y,
                                 anchor='nw', image=self.tk_img)
        
        # Box overlay
        if self.box_start and self.box_end:
            x1 = min(self.box_start[0], self.box_end[0])
            y1 = min(self.box_start[1], self.box_end[1])
            x2 = max(self.box_start[0], self.box_end[0])
            y2 = max(self.box_start[1], self.box_end[1])
            
            # Fill
            self.canvas.create_rectangle(x1, y1, x2, y2,
                                         outline='', fill='lime', stipple='gray25')
            # Border
            self.canvas.create_rectangle(x1, y1, x2, y2,
                                         outline='lime', width=3, fill='')
            # Label
            self.canvas.create_text(x1 + 5, max(y1 - 14, 2),
                                    text='LOGO', fill='lime',
                                    font=('Consolas', 12, 'bold'), anchor='nw')
    
    # ----------------------------------------------------------
    # Actions
    # ----------------------------------------------------------
    
    def _canvas_to_image_coords(self, cx, cy):
        ix = (cx - self.img_offset_x) / self.scale
        iy = (cy - self.img_offset_y) / self.scale
        return ix, iy
    
    def _save_and_next(self):
        if self.box_start is None or self.box_end is None:
            self.status_var.set('⚠️  No box drawn! Click and drag first.')
            return
        
        x1_img, y1_img = self._canvas_to_image_coords(*self.box_start)
        x2_img, y2_img = self._canvas_to_image_coords(*self.box_end)
        x1_img, x2_img = min(x1_img, x2_img), max(x1_img, x2_img)
        y1_img, y2_img = min(y1_img, y2_img), max(y1_img, y2_img)
        
        img_w, img_h = self.current_img.size
        x_center = max(0, min(1, ((x1_img + x2_img) / 2) / img_w))
        y_center = max(0, min(1, ((y1_img + y2_img) / 2) / img_h))
        width    = max(0, min(1, (x2_img - x1_img) / img_w))
        height   = max(0, min(1, (y2_img - y1_img) / img_h))
        
        img_file = self.image_files[self.current_idx]
        label_file = os.path.splitext(img_file)[0] + '.txt'
        label_path = os.path.join(self.label_dir, label_file)
        with open(label_path, 'w') as f:
            f.write(f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")
        
        print(f"✓ [{self._count_annotated()}/{len(self.image_files)}] Saved: {label_file}")
        
        self.current_idx += 1
        self._load_image()
    
    def _skip(self):
        print(f"⏭️  Skipped: {self.image_files[self.current_idx]}")
        self.current_idx += 1
        self._load_image()
    
    def _previous(self):
        if self.current_idx > 0:
            self.current_idx -= 1
            self._load_image()
        else:
            self.status_var.set('⚠️  Already at first image')


# ============================================================
# LAUNCH
# ============================================================

print("Opening annotation window...")
print("  S = Save & Next")
print("  N = Skip")
print("  P = Previous\n")

app = AnnotationApp(WORKSPACE_IMG, WORKSPACE_LBL)

Opening annotation window...
  S = Save & Next
  N = Skip
  P = Previous

✓ [2198/2556] Saved: smsc_paddlechipfdc37c75lfjp.txt
✓ [2199/2556] Saved: smsc_sch5017nwb0608ak3610.txt
✓ [2200/2556] Saved: smsc_slc88b17d0421e771.txt
✓ [2201/2556] Saved: smsc_u8b3343244chcabtw.txt
✓ [2202/2556] Saved: smsc_ultrachip83c790qfp.txt
✓ [2203/2556] Saved: smsc_usb2227nu05.txt
✓ [2204/2556] Saved: smsc_usb2514bid1409a2p10409h2gaatccn.txt
✓ [2205/2556] Saved: smsc_usb2517jzxc1234a2804.txt
✓ [2206/2556] Saved: smsc_usb3315433v3pactw.txt
✓ [2207/2556] Saved: smsc_usx2065jzxd1216a3b19.txt
✓ [2208/2556] Saved: smsc_victory66slc90e66h0035l814.txt
✓ [2209/2556] Saved: sony_cxm4027r929hf5v.txt
✓ [2210/2556] Saved: sst_28ee011.txt
✓ [2211/2556] Saved: steptech_atetechnologyfedex346000200.txt
✓ [2212/2556] Saved: stm_32h750vbt6y7b552vqphlt7936.txt
✓ [2212/2556] Saved: steptech_atetechnologyfedex346000200.txt
✓ [2212/2556] Saved: stm_32h750vbt6y7b552vqphlt7936.txt
✓ [2213/2556] Saved: stm_33990a722.txt
✓ [2214/